##sample echo nest call
rate limit: 2 calls/second
```
http://developer.echonest.com/api/v4/track/profile?api_key=API_KEY&id=musicbrainz:track:2d3e8469-919e-456c-aabc-3a2a4643ff68  

{"response":  
      {"status":  
         {"version": "4.2", "code": 0, "message": "Success"}, 
       "track": 
         {"status": "complete", 
         "song_id": "SOADNSM13E9061E356", 
         "artist": "Amy Winehouse", 
         "foreign_ids": ["musicbrainz:track:2d3e8469-919e-456c-aabc-3a2a4643ff68"], 
         "title": "Tears Dry on Their Own", 
         "foreign_release_ids": ["musicbrainz:release:323b12c3-a7ad-4e2b-b5c8-44b5ef63d5d2"], 
         "foreign_release_id": "musicbrainz:release:323b12c3-a7ad-4e2b-b5c8-44b5ef63d5d2", 
         "catalog": "musicbrainz", 
         "release": "Feel The Love", 
         "foreign_id": "musicbrainz:track:2d3e8469-919e-456c-aabc-3a2a4643ff68", 
         "id": "TRQTUYN13B7BB45482"}}}
         
```

In [1]:
import requests
import time
import pickle
from pprint import pprint

In [2]:
with open("echonestapi1.txt", 'r') as ena:
    api_key = ena.read().strip()

In [3]:
rate_time = 0.501 #3.1

In [4]:
r = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+'&id=musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d')

In [5]:
rdata = r.json()
pprint(rdata)

{u'response': {u'status': {u'code': 0,
                           u'message': u'Success',
                           u'version': u'4.2'},
               u'track': {u'artist': u'Hammock',
                          u'catalog': u'musicbrainz',
                          u'foreign_id': u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d',
                          u'foreign_ids': [u'musicbrainz:track:4bfce89c-b1e6-407f-91ba-e13a81441a7d'],
                          u'foreign_release_id': u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11',
                          u'foreign_release_ids': [u'musicbrainz:release:9f2cecb3-ca9a-4d29-8bcd-a692f32e2f11'],
                          u'id': u'TRRYMSJ13B7BF722BF',
                          u'release': u'More Dead Than Alive (Get Away From the Medicine)',
                          u'song_id': u'SOCGKJR12A58A79D30',
                          u'status': u'complete',
                          u'title': u'I Can Almost See You'}}}


In [6]:
pprint(rdata["response"]["status"]["message"])

u'Success'


In [7]:
#mbid_tracks = {}

In [8]:
linecount = 0
with open("mbids_1.txt", 'r') as mf:
    t0 = time.time()
    for line in mf:
        linecount += 1
        print len(line.split())
        t1 = time.time()
        count = 0
        for mbid in line.split():
            if mbid in mbid_tracks:
                #print "continuing"
                count += 1
                continue
            print count,
            count += 1
            moveon = False
            while not moveon:
                #print "requesting"
                time.sleep(rate_time)
                enr = requests.get('http://developer.echonest.com/api/v4/track/profile?api_key='+api_key+
                                   '&id=musicbrainz:track:'+mbid)
                enrj = enr.json()
                status = enrj["response"]["status"]["message"]
                if status == "Success":
                    #print "true"
                    if "id" in enrj["response"]["track"]:
                        #print "success" #"ids", mbid
                        mbid_tracks[mbid] = enrj["response"]["track"]["id"]
                    moveon = True
                elif status == "The Identifier specified does not exist":
                    #echo nest does not have all mbids mapped to tracks
                    #print status #mbid, status
                    mbid_tracks[mbid] = ""
                    moveon = True
                else:
                    print status
                    time.sleep(3)
        t2 = time.time()
        if t2-t1 > 0.5:
            print "\nline seconds:", t2-t1,
            print "total seconds:", t2-t0, 
            print "total lines:", linecount        
            

1108


NameError: name 'mbid_tracks' is not defined

In [9]:
#print mbid_tracks

In [210]:
print len(mbid_tracks.keys())

 94110


In [197]:
def pickle_stuff(filename, data):
    ''' save file '''
    with open(filename, 'w') as picklefile:
        pickle.dump(data, picklefile)

def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [211]:
pickle_stuff("mbid_track_dict.pkl", mbid_tracks)

In [212]:
with open("mbid_tracks.csv", 'w') as mtf:
    for k, v in mbid_tracks.iteritems():
        mtf.write(k+","+v+"\n")